In [7]:
from bs4 import BeautifulSoup
import urllib.request

#open site
contents = urllib.request.urlopen("https://saint-petersburg.irr.ru/real-estate/apartments-sale/").read()
soup = BeautifulSoup(contents)
mydivs = soup.findAll("div", {"class": "listing__item"})

In [8]:
#url to first pages
yandex_real_estate = urllib.request.urlopen("https://realty.yandex.ru/sankt-peterburg/kupit/kvartira/").read()
url = "https://www.avito.ru/sankt-peterburg/kvartiry?s_trg=1"
avito_real_estate = urllib.request.urlopen("https://www.avito.ru/sankt-peterburg/kvartiry/prodam").read()

In [ ]:
#parse avito with beautifoul soup
avito_soup = BeautifulSoup(avito_real_estate)
url_base = "https://www.avito.ru"
flats = avito_soup.findAll("div", {"class":"item_table-description"})
page_count = 0

data_flats = parse_page(flats, [])

while True:
    try:
        page_count += 1
        print("Page: ", page_count)
        next_page = avito_soup.find("a", {"class":"pagination-page"}).get('href')
        flats = avito_soup.findAll("div", {"class":"item_table-description"})
        data_flats = parse_page(flats, data_flats)
        
        if page_count > 100:
                break
    except Error:
        break

print("finish")
print("data len: ", len(data_flats))



In [230]:
#extract all relevant data from single page
def parse_page(flats, data_flats):
    rent_count = 0
    print("flats on page: ", len(flats))
    for (index, flat) in enumerate(flats):

        print("Flat number", index)

        #price
        print("price:")
        price = price_from_json(flat)
        if(price < 200_000):
            rent_count += 1
            print("Rent found, skip")
            continue
        print(price)
        print()


        #rooms, area, floor level
        print("rooms:")
        description = flat.find("a", {"class": "item-description-title-link"})['title']
        try_rooms = description.split(",")[0][0]
        if try_rooms.isnumeric():
            rooms = int(try_rooms)
        else:
            rooms = 1
        print(rooms)
        print()

        print("area:")
        area = float(description.split(",")[1].split(" ")[1])
        print(area, "m^2")
        print()

        print(description)

        print("floor:")
        floor = description.split(",")[2].split(" ")[1].split("/")[0]
        floors_total = description.split(",")[2].split(" ")[1].split("/")[1]
        print("" + floor + ", total floors: " + floors_total)
        print()

        #subway, distance to subway, adress
        full_adress = flat.find("p", {"class": "address"}).getText()

        adress = ','.join(full_adress.split(",")[1:])
        print("adress:")
        print(adress)
        print()

        subway = full_adress.split(",")[0]
        for (ind,el) in enumerate(subway.split()):
            try:
                float(el)
                subway_station = ' '.join(subway.split()[:ind])
                distance = float(subway.split()[ind])
            except ValueError:
                pass

        print("subway:")
        print(subway_station)
        print()

        if distance < 99:
            distance *= 1000
        print("distance to subway:")
        print(distance, "meters")
        print()

        print("go to next page")
        data_flats.append((price, rooms, area, floor, floors_total, adress, subway_station, distance))
    
    print(rent_count)
    return data_flats
    

In [59]:
#extract price
def price_from_json(flat):
    price = flat.find("div", {"class": "popup-prices"})['data-prices']
    price_ind = price.find('RUB":')
    price_cut = price[price_ind + 5:]
    price_in_rub = price_cut.split(",")[0]
    return int(price_in_rub)

In [202]:
#flat data class combining all relevant data
class Flat:
    
    def __init__(self, flat_price, flat_rooms, flat_area, flat_floor, 
                 flat_floors_total, flat_address, flat_subway_st, flat_distance_to_subway):
        self.flat_price = flat_price
        self.flat_rooms = flat_rooms 
        self.flat_area = flat_area
        self.flat_floor = flat_floor
        self.flat_floors_total = flat_floors_total
        self.flat_address = flat_address
        self.flat_subway_st = flat_subway_st
        self.flat_distance_to_subway = flat_distance_to_subway
        
    def __str__(self):
        return ''.join("adress = "+ flat_address + 
                       ", price = "+ str(flat_price)+ 
                       ", rooms = "+ str(flat_rooms)+
                        ", floor = "+ str(flat_floor)+ 
                       ", area = " + str(flat_area) +
                      ", total_floors = "+ str(flat_floors_total)+ 
                       ", subway = "+ flat_subway_st + 
                       ", distance to subway = " + flat_distance_to_subway)

In [220]:
#
subways = set()
for flat in data_flats:
    subways.add(flat[6])
print(subways)

{'Чкаловская', 'Проспект Просвещения', 'Пионерская', 'Пролетарская', 'Приморская', 'Купчино', 'Проспект Большевиков', 'Елизаровская', 'Ломоносовская', 'Комендантский проспект', 'Василеостровская', 'Ладожская', 'Девяткино', 'Спортивная', 'Московская', 'Чернышевская', 'Автово', 'Звездная', 'Обухово', 'Парнас', 'Достоевская', 'Проспект Ветеранов', 'Новочеркасская', 'Гражданский проспект'}


In [225]:
#clean up parsed data
new_data_flats = []
for flat in data_flats:
    new_flat = (flat[0],flat[1],flat[2],flat[3],flat[4],flat[5].replace('\xa0', '', 2),flat[6],flat[7])
    new_data_flats.append(new_flat)

In [229]:
#write collected data to .csv file as a pandas dataframe
import numpy
import pandas as pd 

a = numpy.asarray(new_data_flats)
pd.DataFrame(a).to_csv("data.csv", header=['price', 'rooms', 'area', 'floor', 'floors_total', 'adress',
                                           'subway_station', 'distance to subway'])